RC1 Calibration Locked – February 2026
- A = 1.5
- Veto threshold = -5.0 (1.5× severe / 0.8× mild)
- Run for USA/PRC or re-score your system

In [15]:
# RC1 Setup – Run once
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Navigate to repo (EDIT THIS PATH to match your Drive location)
%cd /content/drive/MyDrive/15x6-sim  # ← change if repo is elsewhere

!pwd
!ls data/  # should list your CSVs

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/15x6-sim # ← change if repo is elsewhere'
/content/15x6-sim/15x6-sim
/content/15x6-sim/15x6-sim
 35_metrics_ant_colony_baseline.csv
 35_metrics_placeholders.csv
'RC1_placeholder_35-metric_CSV_(USAPRC_calibration).md'


In [17]:
# Cell 1: Imports & config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import fsolve

# Config
import seaborn as sns
sns.set_style("whitegrid")             # or "darkgrid", "white", "ticks"
# sns.set_context("notebook", font_scale=1.2)  # optional: larger fonts
%matplotlib inline

# Cell 2: Load a system (change target to test different ones)
target_system = "USA_2020s"          # or "PRC_2020s", "Ants", etc.
df = pd.read_csv("data/35_metrics_placeholders.csv")

# Quick look
print(df.head())
print(f"\nColumns: {df.columns.tolist()}")

  Metric_ID   Category                                        Description  \
0        A1  Metabolic  Resource acquisition efficiency (GDP/productiv...   
1        A2  Metabolic                        Innovation/R&D output scale   
2        A3  Metabolic                     Energy density/conversion rate   
3        A4  Metabolic                   Raw material sourcing resilience   
4        A5  Metabolic             Labor force participation/adaptability   

   USA_2020s  PRC_2020s                                  Calibration_Notes  
0        8.8        9.4      IMF 2025 growth vs. Total Factor Productivity  
1        9.2        8.8             Patent filings vs. commercialized tech  
2        7.5        9.0  Renewable grid transition vs. manufacturing scale  
3        8.0        9.2   Supply chain diversity vs. state-led procurement  
4        8.5        9.5  Demographic productivity vs. workforce mobiliz...  

Columns: ['Metric_ID', 'Category', 'Description', 'USA_2020s', 'PRC_2020s'

In [39]:
import numpy as np
import pandas as pd

# Load data (adjust path if needed)
df = pd.read_csv("data/35_metrics_placeholders.csv")

# Select system
target = "USA_2020s"  # or "PRC_2020s"

# Extract scores
scores = df[target].astype(float).values

# Metabolic & Governance split
metabolic = scores[0:18]
governance = scores[18:35]

# Rule-13 subset (relative to governance)
rule13_rel_indices = np.array([0,1,2,3,4,5,8])
rule13_values = governance[rule13_rel_indices]

# Base & veto
Y_raw_base = np.mean(governance)
negative_rule13 = rule13_values[rule13_values < 0]
friction_drag = np.mean(negative_rule13) if len(negative_rule13) > 0 else 0.0

friction_threshold = -5.0
veto_multiplier = 1.5 if friction_drag < friction_threshold else 0.8

Y_raw = Y_raw_base - veto_multiplier * abs(friction_drag)

# Projection
A = 1.5
Y_projected = Y_raw * A
Y_clipped = np.clip(Y_projected, -1.0, 1.0)

X = np.clip(np.mean(metabolic) / 10.0, -1.0, 1.0)

# Parasitism %
rule13_norm = rule13_values / 10.0
negative_mask = rule13_norm < 0
parasitism_weighted = np.sum(-rule13_norm[negative_mask]) / len(rule13_norm)
parasitism_pct = parasitism_weighted * 100

# Zone logic
if Y_clipped >= 0.8:
    zone = "Prime Mutualism (Saturated Success)"
    longevity = "Centuries (high-trust baseline)"
elif 0.3 <= Y_clipped < 0.8:
    zone = "Stressed Mutualism"
    longevity = "Decades (drift zone)"
elif -0.29 < Y_clipped < 0.3:
    zone = "Separatrix"
    longevity = "Years (critical instability)"
elif -0.99 <= Y_clipped < -0.29:
    zone = "Active Competition"
    longevity = "Terminal (slow collapse)"
else:
    zone = "Boundary Saturation"
    longevity = "Imminent (kill switch active)"

# Print everything
print(f"--- RC1 Final Lock Diagnostic: {target} ---")
print(f"Metabolic Power (X): {X:.3f}")
print(f"Base Governance Avg: {Y_raw_base:.3f}")
print(f"Rule-13 Friction: {friction_drag:.3f}")
print(f"Veto Multiplier: {veto_multiplier:.1f}")
print(f"Veto-Adjusted Y_raw: {Y_raw:.3f}")
print(f"Projected Y: {Y_projected:.3f}")
print(f"Final Clipped Y: {Y_clipped:.3f}")
print(f"Master Grid Position: ({X:.3f}, {Y_clipped:.3f})")
print(f"Rule-13 Parasitism %: {parasitism_pct:.1f}%")
print(f"Longevity Zone: {zone}")
print(f"Estimated Longevity Window: {longevity}")
print(f"Bifurcation Proximity: {'High' if Y_clipped <= -0.8 else 'Low-Moderate'}")
print(f"Note: Projected Y outside [-1,1] indicates boundary saturation")

--- RC1 Final Lock Diagnostic: USA_2020s ---
Metabolic Power (X): 0.612
Base Governance Avg: 3.765
Rule-13 Friction: -3.500
Veto Multiplier: 0.8
Veto-Adjusted Y_raw: 0.965
Projected Y: 1.447
Final Clipped Y: 1.000
Master Grid Position: (0.612, 1.000)
Rule-13 Parasitism %: 5.0%
Longevity Zone: Prime Mutualism (Saturated Success)
Estimated Longevity Window: Centuries (high-trust baseline)
Bifurcation Proximity: Low-Moderate
Note: Projected Y outside [-1,1] indicates boundary saturation
